In [1]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 82.4 MB/s eta 0:00:00


In [3]:
!pip install transformers

In [2]:
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!pip install datasets

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import torch
import faiss

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)    # Show all rows
pd.set_option('display.max_colwidth', None) # Allow wrapping for long values

In [7]:
with open('/content/drive/MyDrive/logs/sentences.csv', 'r') as file:
    lines = file.readlines()

sentences = pd.Series([line.strip() for line in lines])


In [8]:
print(sentences[0:5])

0     37.152.163.59 accessed /images/zanbil-kharid.png on 2024-08-22T12:38:00 with success.
1                85.9.73.119 accessed /images/next.png on 2024-08-22T12:38:00 with success.
2             85.9.73.119 accessed /images/checked.png on 2024-08-22T12:38:00 with success.
3           37.152.163.59 accessed /images/loading.gif on 2024-08-22T12:38:00 with success.
4    77.245.233.52 accessed /11082/productType/240x180 on 2024-08-22T12:38:00 with success.
dtype: object


In [10]:
sentences_samples = sentences.sample(200000)

# Cümlelerin vektörleştirilmesi

In [9]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
sentence_embeddings = model.encode(sentences_samples.to_list())

In [12]:
sentence_embeddings.shape

(200000, 384)

# Faiss vektör veritabanı kurulumu

In [13]:
import faiss

vector_dim = sentence_embeddings.shape[1]
index = faiss.IndexFlatL2(vector_dim)
index.add(sentence_embeddings)

print(f"number of vectors in the index: {index.ntotal}")

number of vectors in the index: 200000


In [16]:
index_save_path = '/content/drive/MyDrive/t5results/faiss/faiss_index.index'
faiss.write_index(index, index_save_path)

In [ ]:
print(sentences_samples.iloc[0:5])

546495         130.185.76.185 accessed /p35%2Cb36%2Cstexists on 2024-08-26T19:10:00 with success.
2276946         46.209.149.67 accessed /1129/article/100x100 on 2024-08-23T10:29:00 with success.
3930016    83.121.230.196 accessed /31385/productType/120x90 on 2024-08-22T09:38:00 with success.
2282214          37.255.182.61 accessed /430/productTypeType on 2024-08-23T10:31:00 with success.
4942381      5.52.90.13 accessed /61789/productModel/150x150 on 2024-08-24T17:58:00 with success.
dtype: object


# Faiss vektör veritabını denemesi

In [ ]:
query = "Which IP address accessed the /logo URL?"

query_embedding = model.encode([query])

k = 5
distances, indices = index.search(query_embedding, k)

print(f"distances: {distances}")
print(f"indices: {indices}")

print("-"*30)

for i in indices[0]:
  if i != indices[0][0]:
    print(sentences_samples.iloc[i])

distances: [[35.624107 35.650032 36.05517  36.29168  36.544586]]
indices: [[34099 43728  2120  7891 39600]]
------------------------------
5.78.209.192 accessed /logo on 2024-08-25T20:02:00 with success.
192.15.181.80 accessed /logo on 2024-08-25T10:17:00 with success.
192.119.164.75 accessed /logo on 2024-08-25T15:52:00 with success.
192.15.89.215 accessed /logo on 2024-08-22T20:46:00 with success.


# **Q&A PAIRS CREATION**






In [ ]:
data_for_qa = pd.read_csv("/content/drive/MyDrive/logs/samp_without_query.csv")

In [ ]:
data_for_qa.head(2)

Unnamed: 0         client             datetime method  status  size  \
0           1  37.152.163.59  2019-01-22 12:38:27    GET     200   358   
1           2    85.9.73.119  2019-01-22 12:38:27    GET     200  3045   

                                                                                                                                                                                                            referer  \
0  https://www.zanbil.ir/product/29314/%DA%A9%D8%A7%D9%84%D8%B3%DA%A9%D9%87-%D8%AF%D9%88%D9%82%D9%84%D9%88-%D8%AF%D9%84%DB%8C%D8%AC%D8%A7%D9%86-%D9%85%D8%AF%D9%84-%D8%AF%DB%8C%D8%A8%D8%A7-Delijan-Twin-Strollers-   
1                                                                                                                                                                https://znbl.ir/static/bundle-bundle_site_head.css   

                                                                                                          user_agent  \
0                                               Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko   
1  Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36   

   day  hour  minute       referers  browser          os  \
0   22    12      38  www.zanbil.ir  Trident  Windows NT   
1   22    12      38        znbl.ir   Chrome  Windows NT   

                              new_request                   req_name  
0  http://static/images/zanbil-kharid.png  /images/zanbil-kharid.png  
1           http://static/images/next.png           /images/next.png

In [ ]:
def assign_status_category(status):
  if status >= 200 and status < 300:
    return 'success'
  elif status >= 300 and status < 400:
    return 'redirection'
  elif status >= 400 and status < 500:
    return 'client rror'
  elif status >= 500 and status < 600:
    return 'server error'
  else:
    return 'unknown'

data_for_qa['status_category'] = data_for_qa['status'].apply(assign_status_category)

In [ ]:
create_cols = ["client", "req_name", "datetime", "status_category"]
drop_cols = [col for col in data_for_qa.columns if col not in create_cols]

In [ ]:
data_for_qa.drop(drop_cols, axis=1, inplace=True)

In [ ]:
data_for_qa.head(2)

client             datetime                   req_name  \
0  37.152.163.59  2019-01-22 12:38:27  /images/zanbil-kharid.png   
1    85.9.73.119  2019-01-22 12:38:27           /images/next.png   

  status_category  
0         success  
1         success

In [ ]:
data_for_qa.shape

(8365742, 4)

In [ ]:
data_for_qa = data_for_qa.sample(100000)

In [ ]:
data_for_qa.shape

(100000, 4)

In [ ]:
data_for_qa['req_name'] = data_for_qa['req_name'].fillna('')
data_for_qa = data_for_qa[~data_for_qa['req_name'].str.contains('%')]

In [ ]:
data_for_qa.shape

(92692, 4)

In [ ]:
def create_user_specific_qa_pairs(df):
    qa_pairs = []

    for _, row in df.iterrows():
        ip = row['client']
        url = row['req_name']
        datetime = row['datetime']
        status = row['status_category']

        questions = [
            f"When did IP address {ip} access {url}?",
            f"What was the status of the request from IP address {ip} to {url} on {datetime}?",
            f"Which IP addresses accessed {url} on {datetime}?",
            f"What did IP address {ip} access on {datetime}?",
            f"Which IP addresses experienced a {status} when accessing {url}?"
        ]

        for question in questions:
            if "When did IP address" in question:
                answer = f"IP address {ip} accessed {url} on {datetime}."
            elif "status of the request" in question:
                answer = f"The status of the request from IP address {ip} to {url} on {datetime} was {status}."
            elif "Which IP addresses accessed" in question:
                answer = f"IP address {ip} accessed {url} on {datetime}."
            elif "What did IP address" in question:
                answer = f"IP address {ip} accessed {url} on {datetime}."
            elif "Which IP addresses experienced" in question:
                answer = f"IP address {ip} experienced {status} when accessing {url}."
            else:
                answer = "No data available."
            qa_pairs.append({"input": question, "output": answer})

    return qa_pairs


In [ ]:
qa_pairs = create_user_specific_qa_pairs(data_for_qa)

In [ ]:
import random

qa_pairs_sample = random.sample(qa_pairs, 30000)
len(qa_pairs_sample)

30000

# **DATASET OPERATIONS**

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
dataset = Dataset.from_list(qa_pairs_sample)

In [ ]:
dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 30000
})

In [ ]:
new_dataset = DatasetDict({
    'train': dataset
})

print(new_dataset)

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 30000
    })
})


In [ ]:
from sklearn.model_selection import train_test_split

df = new_dataset['train'].to_pandas()

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
train_dataset

Dataset({
    features: ['input', 'output', '__index_level_0__'],
    num_rows: 24000
})

In [ ]:
test_dataset

Dataset({
    features: ['input', 'output', '__index_level_0__'],
    num_rows: 6000
})

# **T5-SMALL MODEL CONFIG AND TRAIN** 

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    inputs = examples['input']
    targets = examples['output']
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding= "max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding= "max_length")
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    output_dir='/content/drive/MyDrive/t5results',
    num_train_epochs=3,
    logging_dir='/content/drive/MyDrive/t5results/logs',
    logging_steps=100,
    load_best_model_at_end=True,
    save_total_limit=5,
    save_steps=500,
    evaluation_strategy="steps",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=9000, training_loss=0.146220581902398, metrics={'train_runtime': 2470.7834, 'train_samples_per_second': 29.141, 'train_steps_per_second': 3.643, 'total_flos': 2436152426496000.0, 'train_loss': 0.146220581902398, 'epoch': 3.0})

In [ ]:
qa_pairs_test = random.sample(qa_pairs, 500)

In [ ]:
input_test = [pair['input'] for pair in qa_pairs_test]
output_test = [pair['output'] for pair in qa_pairs_test]

In [ ]:
def calculate_average_perplexity(model, tokenizer, sentences, device='cpu'):
    model.to(device)
    total_loss = 0.0
    for sentence in sentences:
        # Cümleleri tokenize et
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True).to(device)

        # Model çıktısını al
        with torch.no_grad():
            outputs = model(**inputs, labels=inputs["input_ids"])

        # Kayıp hesapla
        total_loss += outputs.loss.item()

    # Ortalama kaybı ve perplexity hesapla
    average_loss = total_loss / len(sentences)
    average_perplexity = torch.exp(torch.tensor(average_loss)).item()
    return average_perplexity



device = 'cuda' if torch.cuda.is_available() else 'cpu'
average_perplexity = calculate_average_perplexity(model, tokenizer, input_test, device)
print(f"Average Perplexity: {average_perplexity}")


Average Perplexity: 4.786373615264893


In [ ]:
model_save_path = "/content/drive/MyDrive/t5results/model"
model.save_pretrained(model_save_path)

tokenizer_save_path = "/content/drive/MyDrive/t5results/model"
tokenizer.save_pretrained(tokenizer_save_path)

('/content/drive/MyDrive/t5results/model/tokenizer_config.json',
 '/content/drive/MyDrive/t5results/model/special_tokens_map.json',
 '/content/drive/MyDrive/t5results/model/spiece.model',
 '/content/drive/MyDrive/t5results/model/added_tokens.json',
 '/content/drive/MyDrive/t5results/model/tokenizer.json')